In [1]:
import pandas
import numpy as np

# Read the data and load into a python dictionary
files = ['ap_2010.csv', 'class_size.csv', 'demographics.csv', 'graduation.csv', 
         'highschool_directory.csv', 'math_test_results.csv', 'sat_results.csv']

data = {}
for f in files:
    df = pandas.read_csv("datasets/" + f)    #pandas.read_csv("datasets/{0}".format(f))  alternate method
    data[f.replace(".csv", "")] = df


In [2]:
# take a peek at the data
for k,v in data.items():
    print("\n" + k + "\n")
    print(v.head())



math_test_results

      DBN Grade  Year      Category  Number Tested Mean Scale Score Level 1 #  \
0  01M015     3  2006  All Students             39              667         2   
1  01M015     3  2007  All Students             31              672         2   
2  01M015     3  2008  All Students             37              668         0   
3  01M015     3  2009  All Students             33              668         0   
4  01M015     3  2010  All Students             26              677         6   

  Level 1 % Level 2 # Level 2 % Level 3 # Level 3 % Level 4 # Level 4 %  \
0      5.1%        11     28.2%        20     51.3%         6     15.4%   
1      6.5%         3      9.7%        22       71%         4     12.9%   
2        0%         6     16.2%        29     78.4%         2      5.4%   
3        0%         4     12.1%        28     84.8%         1        3%   
4     23.1%        12     46.2%         6     23.1%         2      7.7%   

  Level 3+4 # Level 3+4 %  
0          26 

In [3]:
# make DBN data column where not available
data["class_size"]["DBN"] = data["class_size"].apply(lambda x: "{0:02d}{1}".format(x["CSD"], x["SCHOOL CODE"]), axis=1)
data["highschool_directory"]["DBN"] = data["highschool_directory"]["dbn"]

In [4]:
# Read school survey data
survey1 = pandas.read_csv("datasets/survey_all.txt", delimiter="\t", encoding="windows-1252")
survey2 = pandas.read_csv("datasets/survey_d75.txt", delimiter = "\t", encoding="windows-1252")
survey1["d75"] = False
survey2["d75"] = True
survey = pandas.concat([survey1, survey2], axis=0)

In [5]:
# add the survey data to the dictionary
survey["DBN"] = survey["dbn"]
survey_fields = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", 
                 "saf_t_11", "com_t_11", "eng_t_10", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", 
                 "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]
survey = survey.loc[:, survey_fields]
data["survey"] = survey
survey.shape

(1702, 23)

In [6]:
# Condensing datasets
class_size = data["class_size"]
class_size = class_size[class_size["GRADE "] == "09-12"]
class_size = class_size[class_size["PROGRAM TYPE"] == "GEN ED"]
class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace = True)
data["class_size"] = class_size

In [7]:
demographics = data["demographics"]
demographics = demographics[demographics["schoolyear"] == 20112012]
data["demographics"] = demographics

data["math_test_results"] = data["math_test_results"][data["math_test_results"]["Year"] == 2011]
data["math_test_results"] = data["math_test_results"][data["math_test_results"]["Grade"] == "8"]

data["graduation"] = data["graduation"][data["graduation"]["Cohort"] == "2006"]
data["graduation"] = data["graduation"][data["graduation"]["Demographic"] == "Total Cohort"]

In [8]:
# computing variables
cols = ["SAT Critical Reading Avg. Score", "SAT Math Avg. Score", "SAT Writing Avg. Score"]
for c in cols:
    data["sat_results"][c] = data["sat_results"][c].convert_objects(convert_numeric=True)
data["sat_results"]["sat_score"] = data["sat_results"][cols[0]] + data["sat_results"][cols[1]] + data["sat_results"][cols[2]]

data["highschool_directory"]["lat"] = data["highschool_directory"]["Location 1"].apply(lambda x: x.split("\n")[-1].replace("(", "").replace(")", "").split(", ")[0])  
data["highschool_directory"]["lon"] = data["highschool_directory"]["Location 1"].apply(lambda x: x.split("\n")[-1].replace("(", "").replace(")", "").split(", ")[1])
data["highschool_directory"]["lat"] = pandas.to_numeric(data["highschool_directory"]["lat"])
data["highschool_directory"]["lon"] = pandas.to_numeric(data["highschool_directory"]["lon"])

/Users/babburi/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [9]:
# joining the datasets
flat_data_names = [k for k,v in data.items()]
flat_data = [data[k] for k in flat_data_names]
full = flat_data[0]
for i, f in enumerate(flat_data[1:]):
    name = flat_data_names[i+1]
    print(name)
    print(len(f["DBN"]) - len(f["DBN"].unique()))
    join_type = "inner"
    if name in ["sat_results", "ap_2010", "graduation"]:
        join_type = "outer"
    if name not in ["math_test_results"]:
        full = full.merge(f, on="DBN", how=join_type)

full.shape

sat_results
0
ap_2010
1
demographics
0
class_size
0
survey
0
graduation
0
highschool_directory
0


(374, 174)

In [10]:
cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']
for col in cols:
    full[col] = full[col].convert_objects(convert_numeric = True)
full[cols] = full[cols].fillna(value=0)

/Users/babburi/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


In [11]:
full["school_dist"] = full["DBN"].apply(lambda x: x[:2])
full = full.fillna(full.mean())

In [12]:
# exploring correlations
full.corr()["sat_score"]

Year                                             NaN
Number Tested                           8.127817e-02
SAT Critical Reading Avg. Score         9.868201e-01
SAT Math Avg. Score                     9.726430e-01
SAT Writing Avg. Score                  9.877708e-01
sat_score                               1.000000e+00
AP Test Takers                          5.687940e-01
Total Exams Taken                       5.585421e-01
Number of Exams with scores 3 4 or 5    5.619043e-01
schoolyear                                       NaN
frl_percent                            -7.018217e-01
total_enrollment                        3.668201e-01
ell_num                                -1.535745e-01
ell_percent                            -3.981643e-01
sped_num                                3.486852e-02
sped_percent                           -4.413665e-01
asian_num                               4.748801e-01
asian_per                               5.686267e-01
black_num                               2.7883

In [13]:
# setting the context
import folium
from folium import plugins

schools_map = folium.Map(location = [full["lat"].mean(), full["lon"].mean()], zoom_start=10)
marker_cluster = folium.MarkerCluster().add_to(schools_map)
for name, row in full.iterrows():
    folium.Marker([row["lat"], row["lon"]], popup="{0}: {1}".format(row["DBN"], row["school_name"])).add_to(marker_cluster)
schools_map.save("schools_map.html")
schools_map

ImportError: No module named 'folium'

In [ ]:
schools_heatmap = folium.Map(location=[full["lat"].mean(), full["lon"].mean()], zoom_start=10)
schools_heatmap.add_children(plugins.HeatMap([[row["lat"], row["lon"]] for name, row in full.iterrows()]))
schools_heatmap.save("heatmap.html")
schools_heatmap

In [ ]:
# District level mapping
district_data = full.groupby("school_dist").agg(np.mean)
district_data.reset_index(inplace=True)
district_data["school_dist"] = district_data["school_dist"].apply(lambda x: str(int(x)))
